In [ ]:
import random, os, pickle
import pandas as pd
import numpy as np
import networkx as nx

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm import tqdm

from collections import defaultdict


# Set environment variables for reproducibility and safety
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import precision_score, recall_score, accuracy_score

# 1. Configuration & Seeding
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [ ]:
NAME = 'book' 
INTERACTION_PATH = f'./data/{NAME}_processed_interactions.csv'
GRAPH_PATH = f'./data/{NAME}_processed_graph.csv'

# Hyperparameters
EMBEDDING_DIM = 64
HIDDEN_DIM = 128
LR = 0.001
MAX_STEPS = 3
GAMMA = 0.99 # Discount factor


In [ ]:
class TCKG:
    def __init__(self, tckg_csv_path):
        self.adj_list = defaultdict(list)
        
        print(f"Loading TCKG from {tckg_csv_path}...")
        df_tckg = pd.read_csv(tckg_csv_path, usecols=['head_id', 'relation_id', 'tail_id'])
        
        offset = df_tckg['relation_id'].max() + 1
            
        self.num_relations = offset * 2 # Total relation_id (bidirection)
        print(f"Inverse Relation Offset: {offset}. Total Relation Space: {self.num_relations}")

        data = df_tckg[['head_id', 'relation_id', 'tail_id']].to_numpy() # Using numpy to speedup
        
        for h, r, t in data:
            h, r, t = int(h), int(r), int(t)

            self.adj_list[h].append((r, t))
            self.adj_list[t].append((r + offset, h)) # Ex: r=5 (watched) -> r_inv=105 (watched_by)

        print(f"TCKG Loaded successfully. Graph construction complete.")

    def get_neighbors(self, node_id):
        return self.adj_list[node_id]

    def get_all_nodes(self):
        return list(self.adj_list.keys())

In [ ]:
class TimeAwareRewardFunction(nn.Module):
    def __init__(self, k, entity_embs, relation_embs, interaction_cluster_ids, bias_embs=None, temperature= None):
        """
        Args:
            user_embs (nn.Embedding): Embedding của User (e_u)
            entity_embs (nn.Embedding): Embedding của Item/Entity (e_v)
            relation_embs (nn.Embedding): Embedding của Relation (dùng để lấy V_U)
            interaction_cluster_ids (list or tensor): Danh sách các Relation ID đại diện cho Time Clusters.
                                                      Ví dụ: [20, 21, 22] ứng với interacted_0, interacted_1...
                                                      Đây chính là tập {V_U^1, ..., V_U^L}
            bias_embs (nn.Embedding, optional): Bias của entity (b_v). Nếu None sẽ tự khởi tạo.
        """
        super(TimeAwareRewardFunction, self).__init__()
        
        self.user_embs = user_embs
        self.entity_embs = entity_embs
        self.relation_embs = relation_embs
        
        # Danh sách ID của các cluster tương tác theo thời gian (V_U)
        # Chuyển thành tensor để tính toán song song
        self.register_buffer('cluster_ids', torch.tensor(interaction_cluster_ids, dtype=torch.long))
        
        # Entity Bias (b_v) - Eq (11)
        if bias_embs is None:
            num_entities = entity_embs.num_embeddings
            self.bias_embs = nn.Embedding(num_entities, 1)
            nn.init.zeros_(self.bias_embs.weight) # Khởi tạo bias bằng 0
        else:
            self.bias_embs = bias_embs

        if temperature is None:
            self.temperature = self.entity_embs.embedding_dim ** 0.5
        else:
            self.temperature = temperature



    def calculate_weights(self, history_relation_ids):
        """
        Thực hiện Eq (13): Tính trọng số w_h dựa trên tần suất xuất hiện trong lịch sử.
        
        Args:
            history_relation_ids: (Batch, Max_History_Len) - Chứa relation ID trong quá khứ của user.
            
        Returns:
            weights: (Batch, Num_Clusters) - Vector W_hu
        """
        # 1. So khớp History với Cluster IDs
        # history: (B, H, 1)
        # clusters: (1, 1, L)
        # matches: (B, H, L) -> True nếu relation tại history khớp với cluster ID
        hist_expanded = history_relation_ids.unsqueeze(-1)
        clusters_expanded = self.cluster_ids.view(1, 1, -1)
        
        matches = (hist_expanded == clusters_expanded).float()
        
        # 2. Đếm số lần xuất hiện (Tử số Eq 13)
        # Sum theo chiều History (dim=1) -> (B, L)
        counts = matches.sum(dim=1) 
        
        # 3. Tính độ dài q thực tế (Mẫu số Eq 13)
        # q = tổng số lần xuất hiện của bất kỳ cluster nào trong history (tránh tính padding 0)
        q = counts.sum(dim=1, keepdim=True)
        
        # 4. Normalize để ra trọng số (tránh chia cho 0)
        weights = counts / (q + 1e-9)
        
        return weights

    def forward(self, user_ids, item_ids, history_relation_ids):
        """
        Tính Reward Score g_R(v | u)
        
        Args:
            user_ids: (Batch,)
            item_ids: (Batch,) - Item đích (v_hat) mà Agent dự đoán/dừng lại
            history_relation_ids: (Batch, History_Len) - Lịch sử relation của user
            
        Returns:
            scores: (Batch,) - Điểm reward
        """
        # --- BƯỚC 1: Lấy Embeddings cơ bản ---
        u_e = self.user_embs(user_ids)       # (B, Dim) -> e_u
        v_e = self.entity_embs(item_ids)     # (B, Dim) -> e_v
        v_b = self.bias_embs(item_ids).squeeze(-1) # (B,) -> b_v
        
        # --- BƯỚC 2: Tính Personalized Interaction Relation (Eq 12) ---
        # r_vu^T = W_hu * V_U
        
        # a. Tính weights (B, L)
        weights = self.calculate_weights(history_relation_ids)
        
        # b. Lấy embedding của các cluster V_U^1...L
        # cluster_embs shape: (L, Dim)
        cluster_embs = self.relation_embs(self.cluster_ids)
        
        # c. Tính tổng có trọng số
        # (B, L) x (L, Dim) -> (B, Dim)
        r_interaction = torch.matmul(weights, cluster_embs)
        
        # --- BƯỚC 3: Tính Score (Eq 11 & Final Eq) ---
        # g = (e_u + r_interaction) . e_v + b_v
        
        # Dot product: (e_u + r) * e_v
        query_vector = u_e + r_interaction # (B, Dim)
        dot_product = torch.sum(query_vector * v_e, dim=1) # (B,)
        
        scores = dot_product + v_b # Cộng bias
        
        # 1. Scale Score (Chia cho nhiệt độ)
        scaled_score = scores / self.temperature
        
        # 2. Áp dụng Sigmoid
        rewards = torch.sigmoid(scaled_score)
        
        return rewards


In [ ]:
class TPRecEnvironment(nn.Module):
    def __init__(self, tckg, entity_embeddings, relation_embeddings, max_path_len=3, history_len=3):
        """
        tckg: TCKG object containing adj_list.
        entity_embeddings: nn.Embedding (ll entities)
        relation_embeddings: nn.Embedding (all relations)
        max_path_len: 
        history_len: (k' in paper)
        """
        super(TPRecEnvironment, self).__init__()
        self.tckg = tckg
        self.entity_embs = entity_embeddings
        self.relation_embs = relation_embeddings
        
        self.max_path_len = max_path_len
        self.history_len = history_len
        
        # State tracking
        self.current_entities = None # e_k
        self.current_users = None    # u
        self.histories = None        # h_k (lưu chuỗi relation và entity)
        self.step_counter = 0

    def reset(self, user_ids):
        """
        Initiate status s_0 = (u, u, ∅)
        """

        batch_size = user_ids.size(0)

        self.current_users = user_ids       # Multiple values because of batch_size
        self.current_entities = user_ids    # Initiate status s_0 = (u, u, ∅), the 2nd value is user_ids as well
        
        # History h_k: store (relation, entity)
        # Using torch.zeros for empty ∅
        self.path_history = torch.zeros((batch_size, self.history_len * 2), dtype=torch.long)

        self.step_counter = 0        

        
        return self._get_state_embedding()

    def _get_state_embedding(self):
        """
        State = [User_Emb, Flattened_History_Emb, Current_Entity_Emb]
        Input cho Policy Network sẽ là vector nối dài của 3 thành phần này.
        """
        # 1. User Embedding & Current Entity Embedding
        # Shape: (Batch, Dim)
        u_emb = self.entity_embs(self.current_users)
        e_emb = self.entity_embs(self.current_entities)
        
        # 2. XỬ LÝ HISTORY (Lấy cả Relation và Entity)
        # self.path_history shape: (Batch, history_len * 2) 
        # Cấu trúc: [r1, e1, r2, e2, r3, e3]
        
        # Bước 2a: Tách index của Relation và Entity bằng kỹ thuật Slicing
        # Lấy các cột ở vị trí chẵn (0, 2, 4...) -> Relation Indices
        r_indices = self.path_history[:, 0::2] # Shape: (Batch, history_len)
        
        # Lấy các cột ở vị trí lẻ (1, 3, 5...) -> Entity Indices
        e_indices = self.path_history[:, 1::2] # Shape: (Batch, history_len)
        
        # Bước 2b: Lookup Embedding
        # Shape: (Batch, history_len, Dim)
        r_vecs = self.relation_embs(r_indices) 
        e_vecs = self.entity_embs(e_indices)
        
        # Bước 2c: Kết hợp Relation và Entity tại mỗi bước
        # Cách tốt nhất: Nối (Concat) vector r và e lại với nhau
        # Shape: (Batch, history_len, Rel_Dim + Ent_Dim)
        step_vecs = torch.cat([r_vecs, e_vecs], dim=2)
        
        # Bước 2d: Làm phẳng (Flatten) toàn bộ lịch sử thành 1 vector dài
        # Vì ta muốn giữ thứ tự: bước 1 khác bước 3
        # Shape: (Batch, history_len * (Rel_Dim + Ent_Dim))
        batch_size = step_vecs.size(0)
        h_emb_flat = step_vecs.view(batch_size, -1)

        # 3. KẾT HỢP TẤT CẢ (Concatenate)
        # State = [User (Dim) + History (Len*2*Dim) + Current Entity (Dim)]
        state_vector = torch.cat([u_emb, h_emb_flat, e_emb], dim=1) #[r1, e1, r2, e2, r3, e3]
        
        return state_vector

    def get_pruned_actions(self, epsilon=10):
        """
        Thực hiện Eq (8): Pruning function g_k((r, e_{k+1}) | u)
        Cắt tỉa không gian hành động, chỉ giữ lại top-epsilon neighbors tốt nhất.
        """
        batch_size = self.current_users.size(0)
        valid_actions = []
        
        # Lấy embedding cần thiết cho công thức Eq(8)
        # g_k = (e_u + sum(r_k)) * e_{k+1} + b_{k+1}
        # Lưu ý: sum(r_k) là tổng các relation trong lịch sử (path composition)
        
        u_emb = self.entity_embs(self.current_users) # (B, dim)
        
        # Giả sử ta đã tính được tổng relation embedding của path hiện tại
        # path_rel_sum: (B, dim) 
        
        for i in range(batch_size):
            u_id = self.current_users[i].item()
            curr_node = self.current_entities[i].item()
            
            # Lấy tất cả neighbors của node hiện tại
            neighbors = self.kg.get_neighbors(curr_node) # List các tuple (relation, next_node)
            
            if not neighbors:
                valid_actions.append([]) # Dead end
                continue
                
            # Tách relation và entity node ra
            rels = torch.tensor([n[0] for n in neighbors])
            next_nodes = torch.tensor([n[1] for n in neighbors])
            
            # Tính Score theo Eq (8)
            # Query vector = User + Path_History_Relation
            # Ở đây minh họa đơn giản là User + Relation hiện tại (thực tế cần cộng dồn history)
            r_emb = self.relation_embs(rels)
            next_node_emb = self.entity_embs(next_nodes)
            
            # (e_u + r) * e_{next} (Dot product)
            # score shape: (num_neighbors,)
            query = u_emb[i] + r_emb 
            scores = torch.sum(query * next_node_emb, dim=1) 
            
            # Chọn top epsilon
            k = min(epsilon, len(scores))
            top_scores, top_indices = torch.topk(scores, k)
            
            # Lưu lại danh sách hành động hợp lệ cho user i
            actions_i = []
            for idx in top_indices:
                actions_i.append((rels[idx], next_nodes[idx]))
            valid_actions.append(actions_i)
            
        return valid_actions

    def step(self, actions):
        """
        Transition function (Eq 9): Chuyển trạng thái sang bước k+1
        
        Args:
            actions: List các tuple (relation_id, next_node_id) có độ dài bằng batch_size.
                     Đây là hành động a_k mà Agent vừa chọn.
        
        Returns:
            next_state_emb: Vector trạng thái s_{k+1}
            done: Boolean (True nếu đã đi hết số bước quy định)
        """
        device = self.current_entities.device
        batch_size = len(actions)
        
        # 1. Tách Relation và Entity từ Actions ra thành 2 Tensors riêng biệt
        # actions là list [(r1, e1), (r2, e2)...] -> cần zip lại
        # next_rels: (Batch,)
        # next_ents: (Batch,)
        rels_list = [a[0] for a in actions]
        ents_list = [a[1] for a in actions]
        
        next_relations = torch.tensor(rels_list, dtype=torch.long, device=device)
        next_entities = torch.tensor(ents_list, dtype=torch.long, device=device)
        
        # 2. CẬP NHẬT LỊCH SỬ (History Update - Sliding Window)
        # h_k hiện tại có shape (Batch, history_len * 2)
        # Cấu trúc h_k: [r_{old}, e_{old}, ..., r_{k-1}, e_{k-1}]
        
        # Bước 2a: Tạo cặp (r_k, e_k) mới để nối vào đuôi
        # Shape: (Batch, 1)
        new_r = next_relations.unsqueeze(1)
        new_e = next_entities.unsqueeze(1)
        # Shape: (Batch, 2) -> Mỗi dòng là [r_k, e_k]
        new_entry = torch.cat([new_r, new_e], dim=1)
        
        # Bước 2b: Thực hiện trượt cửa sổ
        # Cắt bỏ 2 phần tử đầu tiên (cũ nhất) của lịch sử hiện tại
        # history_shifted shape: (Batch, (history_len - 1) * 2)
        history_shifted = self.path_history[:, 2:]
        
        # Nối phần mới vào đuôi
        # self.path_history mới shape: (Batch, history_len * 2)
        self.path_history = torch.cat([history_shifted, new_entry], dim=1)
        
        # 3. Cập nhật vị trí hiện tại của Agent (Entity e_k)
        self.current_entities = next_entities
        
        # 4. Kiểm tra điều kiện dừng
        self.step_counter += 1
        done = (self.step_counter >= self.max_path_len)
        
        # 5. Trả về State Embedding mới (s_{k+1})
        return self._get_state_embedding(), done

    def calculate_reward(self, time_aware_scoring_func):
        """
        Reward Function (Eq 10): Soft Reward
        R_K = g_R(e_K | u) / max(g_R(v | u))
        """
        # e_K chính là self.current_entities tại bước cuối cùng
        
        # Tính tử số: Score của entity mà Agent dừng lại
        final_scores = time_aware_scoring_func(self.current_users, self.current_entities)
        
        # Tính mẫu số: Max score có thể đạt được (thường đã pre-calculate hoặc xấp xỉ)
        # Trong thực tế, để nhanh, người ta thường dùng Softmax hoặc Sigmoid của score 
        # thay vì chia cho max exact (vì tìm max tốn kém).
        # Nhưng để đúng công thức Eq 10:
        max_scores = time_aware_scoring_func.get_max_score_per_user(self.current_users)
        
        rewards = final_scores / (max_scores + 1e-9) # Tránh chia cho 0
        
        return rewards

In [ ]:
# ==========================================
# 4. POLICY NETWORK (The Brain)
# ==========================================
class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, hidden_dim):
        super(PolicyNetwork, self).__init__()
        # Input: [State_Emb; Relation_Emb; Next_Node_Emb]
        # Output: Score scalar
        self.fc1 = nn.Linear(state_dim * 3, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1) # Điểm số cho hành động
        self.relu = nn.ReLU()
        
    def forward(self, state_emb, action_embs):
        # state_emb: (1, dim)
        # action_embs (Neighbors): (num_neighbors, dim * 2) gồm [Rel; Next_Node]
        
        # Expand state to match neighbors count
        num_actions = action_embs.shape[0]
        curr_state = state_emb.repeat(num_actions, 1) # (num_neighbors, dim)
        
        # Concat: [Current State, Relation, Next Node]
        x = torch.cat([curr_state, action_embs], dim=1) # (num_neighbors, dim*3)
        
        x = self.relu(self.fc1(x))
        scores = self.fc2(x) # (num_neighbors, 1)
        
        # Softmax để ra xác suất chọn từng neighbor
        probs = F.softmax(scores.view(-1), dim=0)
        return probs

In [ ]:
# ==========================================
# 5. TRAINING LOOP (REINFORCE)
# ==========================================
def train(env, policy_net, train_df, episodes=50):
    optimizer = optim.Adam(policy_net.parameters(), lr=LR)
    
    print("\n>>> Start Training...")
    
    for episode in range(episodes):
        total_reward = 0
        total_loss = 0
        
        # Sample batch user để train cho nhanh (Stochastic)
        batch_samples = train_df.sample(n=256) 
        
        optimizer.zero_grad()
        batch_loss = 0
        
        for _, row in batch_samples.iterrows():
            curr_node = row['head_id']
            target_node = row['tail_id']
            
            episode_log_probs = []
            rewards = []
            
            # --- Walking ---
            done = False
            for step in range(MAX_STEPS):
                # 1. Get State
                state = env.get_state(curr_node, step) # (1, dim)
                
                # 2. Get Valid Actions (Neighbors)
                neighbors, rels, neighbor_ids = env.get_valid_actions(curr_node)
                
                if len(neighbors) == 0:
                    break # Dead end
                
                # 3. Tạo Embedding cho Actions để đưa vào mạng
                neighbor_tensor = torch.tensor(neighbor_ids, dtype=torch.long)
                rel_tensor = torch.tensor(rels, dtype=torch.long)
                
                neigh_embeds = env.node_embeds(neighbor_tensor)
                rel_embeds = env.rel_embeds(rel_tensor)
                action_features = torch.cat([rel_embeds, neigh_embeds], dim=1) # (num_neighbors, dim*2)
                
                # 4. Agent chọn hành động
                probs = policy_net(state, action_features)
                
                # Sampling action dựa trên xác suất (Exploration)
                dist = torch.distributions.Categorical(probs)
                action_idx = dist.sample()
                log_prob = dist.log_prob(action_idx)
                
                episode_log_probs.append(log_prob)
                
                # 5. Execute Action
                next_node = neighbors[action_idx.item()]
                curr_node = next_node
                
                # 6. Check Reward
                if curr_node == target_node:
                    rewards.append(1.0) # Tìm thấy!
                    done = True
                    break
                else:
                    rewards.append(0.0) # Chưa thấy
            
            # --- Tính Loss (Policy Gradient) ---
            # Nếu tìm thấy đích ở bước cuối, phần thưởng lan truyền ngược lại
            # Discounted Return
            R = 0
            returns = []
            for r in reversed(rewards):
                R = r + GAMMA * R
                returns.insert(0, R)
            
            if done: # Chỉ học nếu tìm thấy đích (hoặc có thể phạt nhẹ nếu không tìm thấy)
                total_reward += 1
                for log_prob, R in zip(episode_log_probs, returns):
                    batch_loss -= log_prob * R # Gradient Ascent -> Minimize Negative Reward
        
        # Update Weights
        if batch_loss != 0:
            batch_loss.backward()
            optimizer.step()
            
        if (episode+1) % 5 == 0:
            print(f"Episode {episode+1}/{episodes} | Hit Success: {total_reward}/256 | Batch Loss: {batch_loss:.4f}")
            

In [ ]:
def evaluate(env, policy_net, test_df, top_k=10):
    print("\n>>> Start Evaluation...")
    policy_net.eval() # Chuyển sang chế độ đánh giá (tắt dropout nếu có)
    
    hits = 0
    total_samples = 0
    
    # Lấy mẫu ngẫu nhiên từ Test set để chạy cho nhanh (hoặc chạy hết nếu muốn chính xác)
    test_samples = test_df.sample(n=100) if len(test_df) > 100 else test_df
    
    with torch.no_grad(): # Tắt tính toán gradient để tiết kiệm bộ nhớ
        for _, row in tqdm(test_samples.iterrows(), total=len(test_samples)):
            curr_node = row['head_id']
            target_true = row['tail_id']
            
            # --- Beam Search (Giản lược: Greedy Best-First) ---
            # Để đơn giản, ta cho Agent chọn top-1 đường đi tốt nhất
            
            for step in range(MAX_STEPS):
                state = env.get_state(curr_node, step)
                neighbors, rels, neighbor_ids = env.get_valid_actions(curr_node)
                
                if not neighbors: break 
                
                # Chuẩn bị input cho mạng
                neighbor_tensor = torch.tensor(neighbor_ids, dtype=torch.long)
                rel_tensor = torch.tensor(rels, dtype=torch.long)
                neigh_embeds = env.node_embeds(neighbor_tensor)
                rel_embeds = env.rel_embeds(rel_tensor)
                action_features = torch.cat([rel_embeds, neigh_embeds], dim=1)
                
                # Dự đoán xác suất
                probs = policy_net(state, action_features)
                
                # --- KHÁC BIỆT: Chọn bước đi tốt nhất (Argmax) ---
                # Thay vì random sample, ta chọn nước đi có xác suất cao nhất
                best_action_idx = torch.argmax(probs).item()
                
                next_node = neighbors[best_action_idx]
                curr_node = next_node
                
                # Kiểm tra xem node hiện tại có phải là đích không
                if curr_node == target_true:
                    hits += 1
                    break
            
            total_samples += 1
            
    # Tính HR@1 (Hit Rate) - Tỉ lệ tìm thấy chính xác Item
    acc = hits / total_samples if total_samples > 0 else 0
    print(f"Evaluation Rank 1 (Exact Match): {acc:.4f}")
    
    return acc

In [ ]:
# ==========================================
# MAIN
# ==========================================
if __name__ == "__main__":
    # 1. Load Data
    static_kg, train_df, val_df, test_df = load_and_split_data()
    
    # 2. Init Env (Cần Embedding cho Node/Rel)
    env = KGEnvironment(static_kg, train_df, embedding_dim=EMBEDDING_DIM)
    
    # 3. Init Agent
    policy_net = PolicyNetwork(state_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM)
    
    # 4. Train
    train(env, policy_net, train_df, episodes=50000) # Tăng episodes để thấy reward tăng

    evaluate(env, policy_net, test_df)